<a href="https://colab.research.google.com/github/bgrnaymane/recipe_generator/blob/main/finetuning_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install transformers

In [3]:
import torch
import torch.nn as nn
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

from transformers import Trainer, TrainingArguments

from tqdm.auto import tqdm

import pandas as pd
import numpy as np


In [4]:
model_name = 'gpt2'

In [5]:
model_save_path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model'

In [6]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50260, 768)

In [7]:
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model/tokenizer.json')

In [8]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[50259]

In [9]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs,max_length=256,do_sample=True,pad_token_id=50259)
    print(tokenizer.decode(output[0]))

In [10]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [11]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [12]:
clean = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/data/Cleaned_Indian_Food_Dataset.csv')
clean = clean.sample(frac=1)
clean.reset_index(drop=True,inplace=True)

In [13]:
def print_recipe(idx):
    print(f"{clean['ingredients'][idx]}\n\n{clean['instructions'][idx]}")

In [14]:
def form_string(ingredient,instruction):
    s = f"<|startoftext|>Ingredients:\n{ingredient.strip()}\n\nInstructions:\n{instruction.strip()}<|endoftext|>"
    return s

In [15]:
print(clean.columns)

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'TotalTimeInMins',
       'Cuisine', 'TranslatedInstructions', 'URL', 'Cleaned-Ingredients',
       'image-url', 'Ingredient-count'],
      dtype='object')


In [16]:
data = clean.apply(lambda x:form_string(x['Cleaned-Ingredients'],x['TranslatedInstructions']),axis=1).to_list()

https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e

In [17]:
train_size = 0.85
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [18]:
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=1024,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [19]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [20]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

  0%|          | 0/5047 [00:00<?, ?it/s]

  0%|          | 0/891 [00:00<?, ?it/s]

In [21]:
!pip3 install accelerate -U

In [22]:
!pip3 install transformers[torch]

In [23]:
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=3,
                         save_strategy='no'
                        )

In [24]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [25]:
#from transformers import TrainingArguments

# Define the desired reduced batch size
###reduced_batch_size = 8


#device = torch.device('mps')


#training_args = TrainingArguments(output_dir="test_trainer", use_mps_device=True, per_device_train_batch_size=8,
    #per_device_eval_batch_size=8)

# Modify the TrainingArguments object with reduced batch size
#training_args.per_device_train_batch_size = reduced_batch_size
#training_args.per_device_eval_batch_size = reduced_batch_size



In [26]:

trainer = Trainer(model,
                  args ,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim, scheduler)
                 )


In [27]:
trainer.train()

Step,Training Loss
500,1.017300
1000,0.749700
1500,0.709800
2000,0.678700
2500,0.669700
3000,0.638600
3500,0.637200


TrainOutput(global_step=3786, training_loss=0.7209754577088595, metrics={'train_runtime': 5495.3472, 'train_samples_per_second': 2.755, 'train_steps_per_second': 0.689, 'total_flos': 7912445313024000.0, 'train_loss': 0.7209754577088595, 'epoch': 3.0})

In [28]:
trainer.save_model()

In [29]:
from transformers import pipeline

In [30]:
!pip3 install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.2 MB/s eta 0:00:00


In [32]:
pl = pipeline(task='text-generation',model='/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model')

In [33]:
def create_prompt(ingredients):
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    return s

In [34]:
ingredients = ['Rice,Potatoes,Tomatoes,Spinach,red bell peppers','chicken,tomatoes,aloo,jeera,curry powder']

In [35]:

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Ingredients:
rice
potatoes
tomatoes
spinach
red bell peppers
green bell pepper (capsicum),coriander (dhania) leaves

Instructions:
To begin making the Potato Spinach & Capsicum Rice recipe, heat the oil in a pan.Add the onions and green peppers.
Saute till the onion becomes soft and translucent.Add the potatoes along with a pinch of salt, mix well.
Cook the potatoes till they are soft and cooked.
Turn off the heat and allow the potato to cool.Once cooled, add the spinach, bell peppers, coriander leaves to the potato rice and mix well to combine.
Check the salt and adjust to suit your taste.
Check the spices and adjust to suit your taste.Transfer the Rice to a serving dish, add some chopped spinach and a pinch of cilantro.
Sprinkle some salt, stir well and serve.
<|startoftext|>Ingredients:
chicken
tomatoes
aloo
jeera
curry powder
salt,ginger,coriander (dhania) leaves,cloves garlic,cumin seeds (jeera),green chillies,turmeric powder

Instructions:
To begin making the Cesar

In [36]:
!pip3 install --quiet gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.2 MB/s eta 0:

In [44]:
from transformers import pipeline
import gradio as gr

import warnings
warnings.simplefilter('ignore')

model_path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Project/model'

contrastive_search_config = dict(
    penalty_alpha = 0.5,
    top_k = 5,
    max_new_tokens = 512,
    pad_token_id = 50259
)

model = pipeline('text-generation',model=model_path)

def create_prompt(ingredients):
    ingredients = ','.join([x.strip() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n').lower()
    s = f"<|startoftext|>Ingredients:\n{ingredients}\n\nInstructions:\n"
    return s

def generate(prompt):
    recipe = model(prompt,**contrastive_search_config)[0]['generated_text']
    recipe = recipe.replace('<|startoftext|>','')
    return recipe

def wrapper(ingredients):
    prompt = create_prompt(ingredients)
    recipe = generate(prompt)
    return recipe

intro_html = """
<center><h1>Machine Learning Project: Single work</h1></center>
<center>
<p>Recipe generator based on the fine-tuning of GPT-2 on indian_recipes</p>
</center>
"""

with gr.Blocks() as demo:
    gr.HTML(intro_html)

    ingredients = gr.Textbox(label="Ingredients",
    placeholder='Write ingredients in English and separate them with commas!')

    output = gr.Textbox(label="Recipe",lines=15,)
    greet_btn = gr.Button("Create recipe!")

    gr.Examples(['yellow dal, turmeric, green peas, tomatoes',
                'chicken, soy sauce, tomato sauce, vinegar'],
                inputs=ingredients
            )
    css_code = '''
    body {
        background-image: url("https://picsum.photos/seed/picsum/200/300");
        background-size: cover;
    }
    '''

    greet_btn.click(fn=wrapper, inputs=ingredients, outputs=output)

    #gr.Interface(fn=wrapper, inputs=ingredients, outputs=output, examples=[['']]).launch(share=True, css=css_code)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f32320454f09a0c1c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
